In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
import re
import math
import tensorflow as tf
import os
import shutil
from PIL import Image

In [2]:
import random

In [3]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.utils import shuffle

In [4]:
import ast

In [5]:
dir = r"C:\Users\SPECTRE\Documents\ML\projects\Cats and Dogs"
train_dir = os.path.join(dir, 'images_train')
test_dir = os.path.join(dir, 'images_test')
train_csv = os.path.join(dir, 'train.csv')
test_csv = os.path.join(dir, 'test.csv')

In [6]:
train_dir
train_csv

'C:\\Users\\SPECTRE\\Documents\\ML\\projects\\Cats and Dogs\\train.csv'

Preprocessing DF

In [7]:
trainData = pandas.read_csv(train_csv)
testData = pandas.read_csv(test_csv)

In [8]:
testData.head()

,id
0,img_0966
1,img_6542
2,img_2360
3,img_0763
4,img_6579


In [84]:
m = trainData['breed'].tolist()

In [85]:
en = LabelBinarizer().fit(m)
x = en.transform(m)

In [86]:
newBreed = en.inverse_transform(newBreed)

In [87]:
newBreed

array([13, 14, 21, ..., 23, 14, 15])

In [54]:
z = OneHotEncoder(sparse=False)

In [9]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow import keras

Model

In [10]:
classLabels = 2
breedLabels = 10

In [39]:
inp = keras.layers.Input(shape=(64, 64, 3))
conv1 = keras.layers.Conv2D(64, 5, 1, activation=tf.nn.relu, data_format = 'channels_last')(inp)
conv2 = keras.layers.Conv2D(64,5, 1, activation=tf.nn.relu)(conv1)
max1 = keras.layers.MaxPool2D(pool_size = (2,2),strides= 2, padding= "same")(conv2)
norm1 = keras.layers.BatchNormalization()(max1)
conv3 = keras.layers.Conv2D(64,3, 1, activation=tf.nn.relu)(norm1)
conv4 = keras.layers.Conv2D(64,3, 1, activation=tf.nn.relu)(conv3)
max2 = keras.layers.MaxPool2D(pool_size = (2,2),strides= 2, padding= "same")(conv4)
norm2 = keras.layers.BatchNormalization()(max2)
conv5 = keras.layers.Conv2D(128,3, 1, activation=tf.nn.selu)(norm2)
conv6 = keras.layers.Conv2D(128,3, 1, activation=tf.nn.selu)(conv5)
avg1 = keras.layers.AvgPool2D(pool_size = (2,2),strides= 2, padding= "same")(conv6)
flat = keras.layers.Flatten()(avg1)
fc1 = keras.layers.Dense(1024, activation=tf.nn.relu)(flat)
fc1 = keras.layers.Dropout(0.2)(fc1)
fc2 = keras.layers.Dense(512, activation=tf.nn.relu)(fc1)
fc2 = keras.layers.Dropout(0.2)(fc2)
fc3 = keras.layers.Dense(256, activation=tf.nn.relu)(fc2)
fc3 = keras.layers.Dropout(0.2)(fc3)
fc4 = keras.layers.Dense(128, activation=tf.nn.relu)(fc3)
fc4 = keras.layers.Dropout(0.3)(fc4)

classLabel = keras.layers.Dense(classLabels, activation=tf.nn.softmax, name='class')(fc2)
breedLabel = keras.layers.Dense(breedLabels, activation=tf.nn.softmax, name='breed')(fc4)

model1 = keras.Model(inputs = inp, outputs=[classLabel, breedLabel])

In [40]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 60, 60, 64)   4864        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 56, 56, 64)   102464      conv2d_33[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 28, 28, 64)   0           conv2d_34[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [41]:
model1.compile(optimizer='sgd', loss = {'class':"categorical_crossentropy", 'breed':'categorical_crossentropy'}, metrics=['accuracy'])

Dataset

In [11]:
def load_image(imagePath, size):
    return img_to_array(load_img(imagePath, target_size=(size, size))) / 255.

In [12]:
class ImageSequence(keras.utils.Sequence):
    def __init__(self, df_path, data_path, im_size, batch_size, mode='train'):
        self.df = pandas.read_csv(df_path)
        self.im_size = im_size
        self.batch_size = batch_size
        self.mode = mode
        self.imagePathList = self.df['id'].apply(lambda x: os.path.join(data_path, x + '.jpg')).tolist()
        if self.mode == 'train':
            self.classLabelList = self.df['class_name'].apply(lambda x: float(x)).tolist()
            self.breedLabelList = self.df['breed'].apply(lambda x: float(x)).tolist()
            self.label_encoder = LabelEncoder()
            self.onehot_encoder = OneHotEncoder(sparse=False)
            self.integer_encoded = self.label_encoder.fit_transform(self.classLabelList)
            self.integer_encoded = self.integer_encoded.reshape(len(self.integer_encoded), 1)
            self.classLabelEnc = self.onehot_encoder.fit_transform(self.integer_encoded)
            self.integer1_encoded = self.label_encoder.fit_transform(self.breedLabelList)
            self.integer1_encoded = self.integer1_encoded.reshape(len(self.integer1_encoded), 1)
            self.breedLabelEnc = self.onehot_encoder.fit_transform(self.integer1_encoded)
    
        
    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))
    
    def on_epoch_end(self):
        self.indexes = range(len(self.imagePathList))
        if self.mode == 'train':
            self.indexes = random.sample(self.indexes, k=len(self.indexes))
            
    def get_batch_labels(self, idx): 
        # Fetch a batch of labels
        return [self.classLabelEnc[idx * self.batch_size: (idx + 1) * self.batch_size],
                self.breedLabelEnc[idx * self.batch_size: (idx + 1) * self.batch_size]]

    def get_batch_features(self, idx):
        # Fetch a batch of images
        batch_images = self.imagePathList[idx * self.batch_size: (1 + idx) * self.batch_size]
        return np.array([load_image(im, self.im_size) for im in batch_images])

    def __getitem__(self, idx):
        if self.mode == 'train':
            batch_x = self.get_batch_features(idx)
            batch_y = self.get_batch_labels(idx)
            return batch_x, batch_y
        else:
            batch_x = self.get_batch_features(idx)
            return batch_x

In [42]:
obj = ImageSequence(train_csv, train_dir, im_size = 64, batch_size=44)

C:\Users\SPECTRE\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\SPECTRE\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case yo

In [ ]:
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [31]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('./model.h1', verbose=1, save_best_only=True, mode=min)]

In [43]:
model1.fit_generator(
    generator = obj, 
    steps_per_epoch = 141, 
    epochs = 40,
    verbose = 1,
    callbacks = callbacks
)

Epoch 1/40
141/141 [==============================] - 593s 4s/step - loss: 2.8637 - class_loss: 0.6088 - breed_loss: 2.2548 - class_acc: 0.6813 - breed_acc: 0.1608
Epoch 2/40
141/141 [==============================] - 613s 4s/step - loss: 2.5470 - class_loss: 0.5106 - breed_loss: 2.0364 - class_acc: 0.7577 - breed_acc: 0.2529
Epoch 3/40
141/141 [==============================] - 615s 4s/step - loss: 2.3582 - class_loss: 0.4633 - breed_loss: 1.8949 - class_acc: 0.7846 - breed_acc: 0.3096
Epoch 4/40
141/141 [==============================] - 616s 4s/step - loss: 2.2199 - class_loss: 0.4191 - breed_loss: 1.8008 - class_acc: 0.8111 - breed_acc: 0.3462
Epoch 5/40
141/141 [==============================] - 616s 4s/step - loss: 2.0638 - class_loss: 0.3770 - breed_loss: 1.6868 - class_acc: 0.8316 - breed_acc: 0.3965
Epoch 6/40
141/141 [==============================] - 616s 4s/step - loss: 1.9568 - class_loss: 0.3485 - breed_loss: 1.6083 - class_acc: 0.8464 - breed_acc: 0.4205
Epoch 7/40
141/1

Epoch 22/40
141/141 [==============================] - 638s 5s/step - loss: 0.7100 - class_loss: 0.0418 - breed_loss: 0.6682 - class_acc: 0.9890 - breed_acc: 0.7585
Epoch 23/40
141/141 [==============================] - 634s 4s/step - loss: 0.6681 - class_loss: 0.0380 - breed_loss: 0.6301 - class_acc: 0.9876 - breed_acc: 0.7677
Epoch 24/40
141/141 [==============================] - 634s 4s/step - loss: 0.6608 - class_loss: 0.0492 - breed_loss: 0.6116 - class_acc: 0.9837 - breed_acc: 0.7772
Epoch 25/40
141/141 [==============================] - 628s 4s/step - loss: 0.5785 - class_loss: 0.0306 - breed_loss: 0.5479 - class_acc: 0.9910 - breed_acc: 0.7983
Epoch 26/40
141/141 [==============================] - 634s 4s/step - loss: 0.5068 - class_loss: 0.0270 - breed_loss: 0.4798 - class_acc: 0.9915 - breed_acc: 0.8256
Epoch 27/40
141/141 [==============================] - 655s 5s/step - loss: 0.4639 - class_loss: 0.0202 - breed_loss: 0.4438 - class_acc: 0.9953 - breed_acc: 0.8388
Epoch 28/4

In [44]:
classNameId = {'1': [1, 0], '2':[0, 1]}

In [72]:
testObj = ImageSequence(test_csv, test_dir, im_size = 64, batch_size=32, mode = 'test')

In [73]:
predictions = model1.predict_generator(generator=testObj, verbose=1)

35/35 [==============================] - 36s 1s/step


In [74]:
newClass = list()
newBreed = list()

In [75]:
for i in range(len(testObj.imagePathList)):
    for j in range(2):
        if j==0:
            newClass.append((predictions[j][i])/max(predictions[j][i]))
        elif j==1:
            newBreed.append((predictions[j][i])/max(predictions[j][i]))

In [76]:
newClass = np.array(newClass)

In [77]:
newBreed = np.array(newBreed)

In [78]:
newClass = newClass.astype(int)
newBreed = newBreed.astype(int)

In [79]:
m = trainData['PNEUMONIA']
en = LabelEncoder().fit(m)
x = en.transform(m)
x

array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

In [55]:
n = z.fit_transform(x.reshape(len(x), 1))

C:\Users\SPECTRE\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [80]:
newClass = z.inverse_transform(newClass)

In [81]:
newClass = newClass + 1

In [82]:
newClass

array([[1.],
       [1.],
       [2.],
       ...,
       [2.],
       [1.],
       [1.]])

In [83]:
testData['class_name'] = newClass

In [88]:
testData['breed'] = newBreed

In [89]:
testData['class_name'] = pandas.to_numeric(testData['class_name'], downcast='integer')

In [90]:
testData = testData[['id', 'class_name', 'breed']]

In [91]:
testData.head()

,id,class_name,breed
0,img_0966,1,13
1,img_6542,1,14
2,img_2360,2,21
3,img_0763,1,12
4,img_6579,1,12


In [92]:
xlsxWriter = pandas.ExcelWriter('test2.xlsx', engine='xlsxwriter')

In [93]:
testData.to_excel(xlsxWriter, sheet_name='Sheet1')

In [94]:
xlsxWriter.save()